## LELEC2350 - Simulation of a dispersive propagation
###  Colin Renard (50012000) -  Mathieu Reniers (30322000)

## Theory

Wave paquet/pulse made up by adding frequency components:
$$
E(z,t) = \frac{1}{2\pi} \int_{-\infty}^{\infty} e^{j(wt-kz)} \hat{E}(0,w) dw
$$
As $E$ is real, we can use the conjuguate symetry property:
$$
E(z,t) = \frac{2}{2\pi} \int_{0}^{\infty} \Re \left\{e^{j(wt-kz)} \hat{E}(0,w) \right\} dw
$$

$$
E(z,t) \approx \frac{1}{\pi} \sum_{w} \Re \left\{e^{j(wt-kz)} \hat{E}(0,w) \right\} \Delta w
$$

$$
E(z,t) \approx \frac{1}{\pi}  \sum_{f} \Re \left\{e^{j(2 \pi f t-kz)} \hat{E}(0,f) \right\} 2 \pi \Delta f
$$

$$
E(z,t) \approx 2 \cdot B \cdot  \Re \left\{ \sum_{f} e^{-j(kz - 2\pi f t)} \hat{E}(0,f) \right\}
$$


> In the <font color=#009EDB>non-dispersive</font> case:

$$
k = \frac{w}{c}
$$

> In the <font color=#DB6000>dispersive</font> case, $k$ can be modelled as:

$$
k^2 = \frac{(w^2 + (b^2 - a^2) - j w 2 a)}{c^2}
$$

$\rightarrow$ For a hollow metallic waveguide (WG), $a=0$ and $b = jw_p$:

$$
k = \frac{\sqrt{w^2 + (jw_p)^2 }}{c} = \frac{\sqrt{w - w_p}\sqrt{w + w_p}}{c}
$$



In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from numba import jit
from tqdm.autonotebook import trange, tqdm
%matplotlib qt

#### 1. Useful parameters & Constants

In [13]:
Nt = 200; Nz = int(1e4); Nf = 150       # Resolution in time - space - frequency of the simulation 

z = np.linspace(0,10,Nz)                # z-axis [m]
fc = 3e9                                # Central frequency [Hz]
df = 1e9                                # Deviation frequency [Hz]
f = np.linspace(fc-df,fc+df,Nf)         # Frequency axis [Hz]
B = (fc+df) - (fc-df)                   # Bandwidth [Hz]
t = np.linspace(0,100*(1/fc),Nt)        # Time axis [s]
w,wc = 2*np.pi*f, 2*np.pi*fc            # Pulsation axis & central pulsation
sigma_w = 2*np.pi*0.2*df                # Standard deviation of pulsations

A = (1/(np.sqrt(2*np.pi)*sigma_w)) * np.exp(-(w-wc)**2/(2*(sigma_w**2)))        # Amplitude distribution over frequences (Gaussian)
#A = A*np.sqrt(4*sigma_w*(np.pi**(3/2)))                                       # Normalized

mu = 4*np.pi*1e-7                       # Permeability [H/m]
epsilon = 8.85*1e-12                    # Permittivity [F/m]
c = 1/(np.sqrt(mu*epsilon))             # Speed of light [m/s]

In [14]:
# Plot distribution of frequency (normalized)
plt.figure()
plt.plot(f,A)
plt.fill_between(f, A, 0, alpha=0.2)
plt.xlabel(r"Frequency [Hz]")
plt.ylabel(r"$\hat{E}(0,w)$")
plt.title("Frequency distribution")
plt.show()
plt.savefig("frequency_distribution.pdf")
integral = np.trapz(A*A, x=w, dx=w[1]-w[0])
print("Integral / 2pi = ", integral/(2*np.pi))



Integral / 2pi =  3.5727722752278186e-11


#### 2. Non-dispersive case

In [5]:
k = w/c # Wavenumbers                    

@jit(nopython=True,error_model="numpy") # Optimization
def compute_E_non_dispersive(z,t):
    E = np.zeros(len(z))
    for i in range(len(z)):
        E[i] = 2 * B *  np.real(np.sum(A*np.exp(-1j*k*z[i])*np.exp(1j*w*t))) # (1/pi) dw = 2 df
    return E

In [6]:
E_non_dispersive = np.zeros((Nt,Nz))
for i in tqdm(range(Nt), "time"):
    E_non_dispersive[i] = compute_E_non_dispersive(z,t[i])

time:   0%|          | 0/200 [00:00<?, ?it/s]

#### 3. Dispersive case (WG)

In [7]:
wc_disp = 2*np.pi * 2e9                                             # Cutoff frequency of the waveguide
k_disp = (1/c) * np.sqrt((w+wc_disp)) * np.sqrt((w-wc_disp))        # Wavenumver (f) 

@jit(nopython=True,error_model="numpy") # Optimization
def compute_E_dispersive(z,t):
    # Computes s for an array z at a particular time t
    E = np.zeros(len(z))
    for i in range(len(z)):
        E[i] =  2 * B * np.real(np.sum(A*np.exp(-1j*k_disp*z[i])*np.exp(1j*w*t)))
    return E

In [8]:
E_dispersive = np.zeros((Nt,Nz))
for i in tqdm(range(Nt), "time"):
    E_dispersive[i] = compute_E_dispersive(z,t[i])

time:   0%|          | 0/200 [00:00<?, ?it/s]

#### 4. Animation

In [9]:
fig, ax = plt.subplots()
ax.set_xlabel("z [m]")
ax.set_ylabel("Electric Field [V/m]")
ax.set_title("Propagation in a dispersive and in a non-dispersive medium")
ax.grid()

line, = ax.plot(z, E_non_dispersive[0], label="Non dispersive medium")
line2, = ax.plot(z, E_dispersive[0], label="Dispersive medium (WG)")
ax.legend()

def anim(i):       
    #line.set_ydata(compute_s(z,t[i]))  # update the data
    line.set_ydata(E_non_dispersive[i])
    line2.set_ydata(E_dispersive[i])
    return line,

ani = animation.FuncAnimation(fig, anim, frames=len(t),interval=50, repeat=True ,blit=False)
plt.show()

#writer = animation.PillowWriter(fps=15, metadata=dict(artist='Me'),bitrate=1800)
#ani.save('scatter.gif', writer=writer)

---

## Normalization of the Energy


Distribution over frequencies:
$$
\hat{E}(0,w) = \frac{1}{\sigma_w \sqrt{2\pi}} e^{-\frac{(w-w_c)^2}{2\sigma_{w}^{2}}}
$$

We know from statistics that: 
$$
\int_{w} \hat{E}(0,w) dw = 1
$$

The Parseval theorem says that energy of a signal $f$ is:
$$
\mathcal{E}_{f} = \int_{-\infty}^{\infty} |f(t)|^{2} dt  = \frac{1}{2\pi }\int_{-\infty}^{\infty} |F(w)|^{2} dw
$$

We have:


$$
\mathcal{E}_{f} = \frac{1}{2\pi }\int_{-\infty}^{\infty} |\hat{E}(0,w)|^{2} dw = \frac{1}{2\pi }\int_{-\infty}^{\infty} \left|\frac{1}{\sigma_w \sqrt{2\pi}} \exp \left({-\frac{(w-w_c)^2}{2\sigma_{w}^{2}}} \right) \right|^{2} dw
$$

$$
\mathcal{E}_{f} = \frac{1}{2\pi} \cdot \frac{1}{\sigma_w \sqrt{2\pi}} \int_{-\infty}^{\infty} \frac{1}{\sigma_w \sqrt{2\pi}} \exp \left( {-2\cdot \frac{(w-w_c)^2}{2\sigma_{w}^{2}}} \right)  dw
$$

$$
\mathcal{E}_{f} = \frac{1}{2\pi} \cdot \frac{1}{\sigma_w \sqrt{2\pi}} \int_{-\infty}^{\infty} \frac{1}{\sigma_w \sqrt{2\pi}} \exp \left( {- \frac{(w-w_c)^2}{\sigma_{w}^{2}}} \right)  dw
$$

We define $\sigma_{w,bis} = \sigma_{w}/\sqrt{2}$:

$$
\mathcal{E}_{f} = \frac{1}{2\pi} \cdot \frac{1}{\sigma_{w} \sqrt{2\pi}} \int_{-\infty}^{\infty} \frac{1}{ \sqrt{2} \sigma_{w,bis} \sqrt{2\pi}} \exp {- \frac{(w-w_c)^2}{2 \sigma_{w,bis}^{2}}}  dw
$$

$$
\mathcal{E}_{f} = \frac{1}{2\pi} \cdot \frac{1}{2 \sigma_{w} \sqrt{\pi}} \underbrace{\int_{-\infty}^{\infty} \frac{1}{ \sigma_{w,bis} \sqrt{2\pi}} \exp {- \frac{(w-w_c)^2}{2 \sigma_{w,bis}^{2}}}  dw}_{= 1}
$$

$$
\mathcal{E}_{f} = \frac{1}{2\pi} \cdot \frac{1}{2 \sigma_{w} \sqrt{\pi}}
$$

We want to normalize the energy of the field, we impose $\mathcal{E}_{f} = 1$, so we will work with
$$
\hat{E}_{\text{normalized}}(0,w) = \sqrt{ 4 \sigma_w \pi^{3/2} } \hat{E}(0,w)
$$